In [ ]:
CREATE STREAMING LIVE VIEW yellow_tripdata_view

SELECT
  ride_uuid
  , 'yellow' taxi_type
  , vendorid::int
  , COALESCE(passenger_count, '1')::double passenger_count
  , total_amount::double
  , tpep_pickup_datetime::timestamp pickup_datetime
  , tpep_dropoff_datetime::timestamp dropoff_datetime
  , date_partition
FROM
  STREAM(main.bronze.yellow_tripdata)
;

In [ ]:
CREATE STREAMING LIVE VIEW green_tripdata_view

SELECT
  ride_uuid
  , 'green' taxi_type
  , vendorid::int
  , COALESCE(passenger_count, '1')::double passenger_count
  , total_amount::double
  , lpep_pickup_datetime::timestamp pickup_datetime
  , lpep_dropoff_datetime::timestamp dropoff_datetime
  , date_partition
FROM
  STREAM(main.bronze.green_tripdata)
;

In [ ]:
CREATE STREAMING LIVE TABLE nyc_tripdata_view

SELECT
  *
FROM
  STREAM(LIVE.yellow_tripdata_view)

UNION ALL

SELECT
  *
FROM
  STREAM(LIVE.green_tripdata_view)
;

In [ ]:
CREATE STREAMING LIVE TABLE nyc_tripdata_silver
PARTITIONED BY (date_partition)
TBLPROPERTIES("Quality" = "Silver")
COMMENT 'Green and Yellow Taxi Data From NYC open data.'

SELECT
  *
  , CURRENT_TIMESTAMP() updated_at_utc
FROM
  STREAM(LIVE.nyc_tripdata_view)
WHERE TRUE 
  AND total_amount >= 0
;